<a href="https://colab.research.google.com/github/aliebi/sup_fact/blob/main/support_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Mar 13 15:24:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

In [3]:
!pip install torch torchvision
!pip install transformers
!pip install pandas
!pip install numpy

In [4]:
!wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
!wget http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_distractor_v1.json

--2021-03-13 15:25:02--  http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
Resolving curtis.ml.cmu.edu (curtis.ml.cmu.edu)... 128.2.204.193
Connecting to curtis.ml.cmu.edu (curtis.ml.cmu.edu)|128.2.204.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566426227 (540M) [application/json]
Saving to: ‘hotpot_train_v1.1.json.2’

hotpot_train_v1.1.j 100%[===================>] 540.19M  38.5MB/s    in 14s     

2021-03-13 15:25:16 (37.4 MB/s) - ‘hotpot_train_v1.1.json.2’ saved [566426227/566426227]

--2021-03-13 15:25:16--  http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_distractor_v1.json
Resolving curtis.ml.cmu.edu (curtis.ml.cmu.edu)... 128.2.204.193
Connecting to curtis.ml.cmu.edu (curtis.ml.cmu.edu)|128.2.204.193|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46320117 (44M) [application/json]
Saving to: ‘hotpot_dev_distractor_v1.json.2’

hotpot_dev_distract 100%[===================>]  44.17M  27.3MB/s    in 1.6s    



In [5]:
import pandas as pd
import nltk
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
nltk.download('punkt')


new_data = pd.DataFrame(columns=['id','question','label','title','context','answer','type','level','position'])


print(1)
def convert(input):
    df = pd.read_json(input)

    print(type(df))
    row_list = []
    for index, row in df.iterrows():
        sup = []
        sup_sent = []
        for i in row['supporting_facts']:
            sup.append(i[0])
            sup_sent.append((i[0], i[1]))

        for i in row['context']:

            dic = {}
            dic['id'] = row['_id']
            dic['question'] = row['question']
            dic['title'] = i[0]

            dic['answer'] = row['answer']
            dic['type'] = row['type']
            dic['level'] = row['level']

            position = []
            for j in sup_sent:

                if i[0] == j[0]:
                    position.append(j[1])

            dic['position'] = position
            str = ''
            for j in i[1]:
                str = str + j

            dic['context'] = str

            if i[0] in sup:
                dic['label'] = True
            else:
                dic['label'] = False

            row_list.append(dic)

    xd = pd.DataFrame(row_list)

    return xd

dev = convert('/content/hotpot_dev_distractor_v1.json')[:1000]
train = convert('/content/hotpot_train_v1.1.json')[:10000]


print(2)

dev = dev[dev['label'] != False]
train = train[train['label'] != False]


from nltk import tokenize
def find_position(input):

  all_rows = []
  for _,i in input.iterrows():
    sentences = tokenize.sent_tokenize(i['context'])
    dic = i
    All_position = []
    for j,k in enumerate(sentences):
      if j in i['position']:
        All_position.append(1)
      else :
        All_position.append(0)

    dic['all_position'] = All_position
    all_rows.append(dic)

  return pd.DataFrame(all_rows)


dev = find_position(dev)
train = find_position(train)

print(3)
def change_dataset(dataset1):
    list_of_rows = []
    for i in range(0, len(dataset1)-1, 2):
        dictionary = {}

        dictionary['answer'] = dataset1.iloc[i]['answer']

        dictionary['context'] = dataset1.iloc[i]['context'] + " " + dataset1.iloc[i + 1]['context']
        dictionary['all_position'] = dataset1.iloc[i]['all_position'] + dataset1.iloc[i + 1]['all_position']
        dictionary['question'] = dataset1.iloc[i]['question']

        list_of_rows.append(dictionary)

    return pd.DataFrame(list_of_rows)


train = change_dataset(train)
dev = change_dataset(dev)

print(3)
def find_gold_sentence(input):
    list_of_rows = []
    for _, i in input.iterrows():
        dictionary = {}
        dictionary = i
        sentences = tokenize.sent_tokenize(i['context'])
        if i['answer'] == 'yes' or i['answer'] == 'no':
            gold = ''
        else:
            for j in sentences:
                if i['answer'] in j:
                    gold = j
                    break
        dictionary['gold_sentenc'] = gold
        list_of_rows.append(dictionary)

    return pd.DataFrame(list_of_rows)


dev = find_gold_sentence(dev)
train = find_gold_sentence(train)

print(4)
def change_format(input):
    list_of_rows = []
    for _, i in input.iterrows():
        sentences = tokenize.sent_tokenize(i['context'])
        for j, k in enumerate(sentences):
            dictionary = {}
            dictionary['label'] = i['all_position'][j]
            dictionary['question'] = i['question']
            dictionary['answer'] = i['answer']
            dictionary['gold_sentenc'] = i['gold_sentenc']
            dictionary['context'] = k
            list_of_rows.append(dictionary)

    return pd.DataFrame(list_of_rows)


dev = change_format(dev)
train = change_format(train)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
1
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
2
3
3
4


In [6]:
len(dev)

631

In [7]:
train =train.rename({'gold_sentenc': 'sentence1', 'context': 'sentence2'}, axis=1)  # new method
dev =dev.rename({'gold_sentenc': 'sentence1', 'context': 'sentence2'}, axis=1)  # new method

In [8]:
train.loc[0]['sentence1']

"Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century."

In [9]:
! pip install transformers

In [10]:
model_checkpoint = "roberta-base" 
batch_size = 1

In [11]:
! pip install datasets


In [12]:
from datasets import load_dataset, load_metric

In [13]:
actual_task = 'mrpc'
from datasets import load_dataset, load_metric
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [ ]:
del dataset

In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [14]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.59375, 'f1': 0.5357142857142857}

In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [69]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.",truncation=True)

{'input_ids': [0, 31414, 6, 42, 65, 3645, 328, 2, 2, 2409, 42, 3645, 1411, 19, 24, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
def preprocess_function(examples):

    return tokenizer(examples['sentence1']+" "+examples['question'], examples['sentence2'],truncation=True)
    

In [17]:
train_data = []
for _,i in train.iterrows():
    tmp = preprocess_function(i)
    tmp['label'] = i['label']
    train_data.append(tmp)

dev_data = []
for _,i in dev.iterrows():
    tmp = preprocess_function(i)
    tmp['label'] = i['label']
    dev_data.append(tmp)

In [72]:
train_data[0]

{'input_ids': [0, 28678, 18, 10202, 36, 1366, 3305, 2383, 1366, 3761, 43, 21, 41, 470, 17205, 675, 3569, 1027, 11, 3378, 11, 5, 753, 212, 3220, 4, 6834, 4320, 21, 554, 78, 9411, 18, 10202, 50, 1234, 13, 2691, 116, 2, 2, 28678, 18, 10202, 36, 1366, 3305, 2383, 1366, 3761, 43, 21, 41, 470, 17205, 675, 3569, 1027, 11, 3378, 11, 5, 753, 212, 3220, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 1}

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [19]:
metric_name = "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [21]:
task ='mrpc'
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=dev_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,0.917800,0.736212,0.858954,0.803532,5.550700,113.679000
2,0.793900,0.716549,0.868463,0.825996,5.525100,114.206000
3,0.479400,0.781765,0.882726,0.842553,5.548500,113.725000
4,0.444400,0.857500,0.876387,0.835443,5.551200,113.669000
5,0.332900,0.866541,0.871632,0.837675,5.592400,112.832000


TrainOutput(global_step=32395, training_loss=0.5691388329931166, metrics={'train_runtime': 1725.2273, 'train_samples_per_second': 18.777, 'total_flos': 2156868453774300, 'epoch': 5.0})

In [23]:
trainer.evaluate()

{'epoch': 5.0,
 'eval_accuracy': 0.8827258320126783,
 'eval_f1': 0.8425531914893617,
 'eval_loss': 0.7817652225494385,
 'eval_runtime': 5.6115,
 'eval_samples_per_second': 112.447}

In [ ]:
len(train)

6479

In [24]:
tmp = trainer.predict(dev_data)

In [25]:
tmp.predictions

array([[-3.1204455,  3.2038207],
       [ 3.645518 , -3.4762647],
       [-2.9388738,  3.022535 ],
       ...,
       [-3.1216776,  3.2067878],
       [ 3.646599 , -3.473487 ],
       [-3.0942001,  3.1660218]], dtype=float32)

In [26]:
y = np.argmax(tmp.predictions, axis=1).flatten()

In [ ]:
len(y)

631

In [27]:
dev_2 = dev.copy()
dev_2 ['predict'] = y 

In [28]:
dev_2.to_csv('predict_sup')

In [29]:
dev_2 = pd.read_csv('/content/predict_sup')
dev_2.head()

,Unnamed: 0,label,question,answer,sentence1,sentence2,predict
0,0,1,Were Scott Derrickson and Ed Wood of the same ...,yes,NaN,"Scott Derrickson (born July 16, 1966) is an Am...",1
1,1,0,Were Scott Derrickson and Ed Wood of the same ...,yes,NaN,"He lives in Los Angeles, California.",0
2,2,0,Were Scott Derrickson and Ed Wood of the same ...,yes,NaN,He is best known for directing horror films su...,1
3,3,1,Were Scott Derrickson and Ed Wood of the same ...,yes,NaN,"Edward Davis Wood Jr. (October 10, 1924 – Dece...",1
4,4,1,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...","Shirley Temple Black (April 23, 1928 – Februar...",0


In [30]:
dev.head(10)

,label,question,answer,sentence1,sentence2
0,1,Were Scott Derrickson and Ed Wood of the same ...,yes,,"Scott Derrickson (born July 16, 1966) is an Am..."
1,0,Were Scott Derrickson and Ed Wood of the same ...,yes,,"He lives in Los Angeles, California."
2,0,Were Scott Derrickson and Ed Wood of the same ...,yes,,He is best known for directing horror films su...
3,1,Were Scott Derrickson and Ed Wood of the same ...,yes,,"Edward Davis Wood Jr. (October 10, 1924 – Dece..."
4,1,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...","Shirley Temple Black (April 23, 1928 – Februar..."
5,1,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...","As an adult, she was named United States ambas..."
6,1,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...",Kiss and Tell is a 1945 American comedy film s...
7,0,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...","In the film, two teenage girls cause their res..."
8,0,What government position was held by the woman...,Chief of Protocol,"As an adult, she was named United States ambas...",The parents' bickering about which girl is the...
9,1,"What science fantasy young adult series, told ...",Animorphs,The Hork-Bajir Chronicles is the second compan...,The Hork-Bajir Chronicles is the second compan...


In [31]:
q = 'Were Scott Derrickson and Ed Wood of the same nationality?'

t1_label= ''
t2_label= []

t1_predict= ''
t2_predict= []
for i,j in dev.iterrows():

  if j['question'] != q:
    t2_label.append(t1_label)
    t1_label = ''

    t2_predict.append(t1_predict)
    t1_predict = ''

    q = j['question']
    

  t1_label = t1_label+ str(dev_2.loc[i]['label'])
  t1_predict = t1_predict + str(dev_2.loc[i]['predict'])




In [ ]:
t2_label

['1001',
 '11100',
 '1110110',
 '11',
 '10100',
 '100100',
 '110',
 '100100',
 '1010010',
 '1010',
 '101100',
 '11100',
 '01110',
 '0010010',
 '001110',
 '1000100',
 '0111',
 '11',
 '100010',
 '1100',
 '1011',
 '0100001',
 '110001100',
 '011',
 '1011',
 '1001',
 '0100000100',
 '10000100000',
 '110000100000',
 '101',
 '100000010000',
 '11000',
 '10010',
 '100100',
 '111100',
 '0000011000000000',
 '1000100',
 '10001000000',
 '1100',
 '11',
 '11001',
 '00010010',
 '100100',
 '01001',
 '10001010',
 '111100',
 '110010000000',
 '10000100',
 '1000100',
 '000001000010000',
 '1011',
 '01000001',
 '10010',
 '00100001',
 '1010001001',
 '110110',
 '100100000000',
 '10000010000',
 '001010',
 '1010',
 '001000101',
 '100010010',
 '100010',
 '10100001',
 '0100010',
 '011100',
 '011',
 '010000001',
 '01000001',
 '01010',
 '0100100',
 '10000100',
 '1001',
 '1001000',
 '100001100',
 '101101',
 '110',
 '110001',
 '110000',
 '110',
 '0011',
 '100000100',
 '10000100100',
 '10100',
 '111',
 '100000100',
 '01

In [ ]:
t2_predict

['1101',
 '01000',
 '1010100',
 '11',
 '10100',
 '010100',
 '100',
 '101100',
 '1010010',
 '1010',
 '110000',
 '10100',
 '10100',
 '1000010',
 '000010',
 '1001100',
 '1101',
 '11',
 '100100',
 '1101',
 '1010',
 '0100000',
 '100001100',
 '101',
 '1011',
 '1001',
 '0100000000',
 '10000000000',
 '011000100000',
 '101',
 '100000010010',
 '11000',
 '10100',
 '100100',
 '101000',
 '1000001000000000',
 '1000100',
 '10000000000',
 '1100',
 '11',
 '11000',
 '10010010',
 '101000',
 '11100',
 '00000010',
 '100100',
 '100010000000',
 '10100100',
 '1000100',
 '000000100010000',
 '1001',
 '11000100',
 '10010',
 '10000101',
 '1010001000',
 '100110',
 '100100000000',
 '10000010000',
 '111010',
 '1010',
 '000001100',
 '001010000',
 '100010',
 '00100000',
 '0000110',
 '010000',
 '101',
 '010001000',
 '01000000',
 '00010',
 '0100010',
 '10010100',
 '0001',
 '1011000',
 '100001000',
 '100001',
 '110',
 '000001',
 '111000',
 '110',
 '1011',
 '010000100',
 '00000100000',
 '10100',
 '111',
 '100000100',
 '11

In [32]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):


    return s


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1
def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))

def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    
    score = metric_fn(prediction, ground_truths)
    scores_for_ground_truths.append(score)

    return scores_for_ground_truths[0]

def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      # print(ground_truths,"   ",prediction)
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total


    return {'exact_match': exact_match, 'f1': f1}
            

In [33]:
evaluate(t2_label,t2_predict)

{'exact_match': 52.525252525252526, 'f1': 52.525252525252526}

In [34]:
cnt = 0
for i,j in zip(t2_label,t2_predict):

  if i == j :
      cnt +=1


cnt /len(t2_label)

0.5252525252525253

In [35]:
for _,i in dev_2.iterrows():
  print('Sen1 :',i['sentence1'])
  print('Sen2 :',i['sentence2'])
  print('label :',i['label'])
  print('predict :',i['predict'])
  print('*'*20)

Sen1 : nan
Sen2 : Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
label : 1
predict : 1
********************
Sen1 : nan
Sen2 : He lives in Los Angeles, California.
label : 0
predict : 0
********************
Sen1 : nan
Sen2 : He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."
label : 0
predict : 1
********************
Sen1 : nan
Sen2 : Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.
label : 1
predict : 1
********************
Sen1 : As an adult, she was named United States ambassador to Ghana and to Czechoslovakia and also served as Chief of Protocol of the United States.
Sen2 : Shirley Temple Black (April 23, 1928 – February 10, 2014) was an American actress, singer, dancer, businesswoman, and diplomat who was Hollywood's 